In [1]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


In [2]:
# Import and read the charity_data.csv.

import pandas as pd 
motor_df = pd.read_csv('Resources/Motor_Vehicle_Crashes_-_Vehicle_Information__Three_Year_Window.csv')
motor_df.head()

,Year,Case Vehicle ID,Vehicle Body Type,Registration Class,Action Prior to Accident,Type / Axles of Truck or Bus,Direction of Travel,Fuel Type,Vehicle Year,State of Registration,Number of Occupants,Engine Cylinders,Vehicle Make,Contributing Factor 1,Contributing Factor 1 Description,Contributing Factor 2,Contributing Factor 2 Description,Event Type,Partial VIN
0,2018,15232120,VAN TRUCK,AGRICULTURAL TRUCK,Parked,Not Entered,North,Gas,2015.0,NY,0.0,4.0,NISSA,HUMAN,Not Entered,HUMAN,Not Entered,Not Entered,3N6CM0KN2FK713460
1,2018,15232535,SUBURBAN,PASSENGER OR SUBURBAN,Making U Turn,Not Entered,South,Gas,2015.0,NY,1.0,6.0,HONDA,HUMAN,Not Entered,HUMAN,Unknown,Not Entered,5J6TF2H55FL001556
2,2018,15232536,SEDAN,Not Entered,Going Straight Ahead,Not Entered,North,Not Entered,NaN,PA,1.0,NaN,NaN,HUMAN,Not Entered,HUMAN,Unknown,Not Entered,NaN
3,2018,15232557,4 DOOR SEDAN,PASSENGER OR SUBURBAN,Parked,Not Entered,South,Gas,2013.0,NY,5.0,4.0,NISSA,HUMAN,Passenger Distraction,VEHICLE,Other*,Not Entered,1N4AL3AP8DC229156
4,2018,15232556,SUBURBAN,PASSENGER OR SUBURBAN,Going Straight Ahead,Not Entered,South,Gas,2009.0,NY,2.0,8.0,CADIL,HUMAN,Not Applicable,HUMAN,Not Applicable,Not Entered,3GYFK22209G213337


In [3]:
len(motor_df)

1648853

In [4]:
# Drop all NA from rows

motor_df = motor_df.dropna()

In [5]:
# Get Names for indexs for which Contributing Factor  1 Description 1 is Not Entered

not_entered = motor_df[motor_df['Contributing Factor 1 Description'] == 'Not Entered'].index

In [6]:
# Drop these rows

motor_df.drop(not_entered, inplace = True)

In [7]:
# Get Names for indexs for which Contributing Factor  1 Description 1 is Not applicable

not_app = motor_df[motor_df['Contributing Factor 1 Description'] == 'Not Applicable'].index

In [8]:
# Drop these rows

motor_df.drop(not_app, inplace = True)

In [9]:
# Get Names for index for which Contributing Factor 1 Description is Unknown

unknown = motor_df[motor_df['Contributing Factor 1 Description'] == 'Unknown'].index

In [10]:
# Drop these rows

motor_df.drop(unknown, inplace = True)

In [11]:
len(motor_df)

418971

In [13]:
motor_df.to_csv('Resources/Motor_Vehicle_Crashes_clean.csv')

In [14]:
# Direction of Travel

motor_df['Direction of Travel'].value_counts()

East              94530
West              94191
South             92360
North             92221
Northeast         11517
Southwest         11124
Southeast         11071
Northwest         11061
Unknown             871
Not Applicable       25
Name: Direction of Travel, dtype: int64

In [15]:
# get unknown values in direction of travel

unknown2 = motor_df[motor_df['Direction of Travel'] == 'Unknown'].index

In [16]:
# Drop these rows

motor_df.drop(unknown2, inplace = True)

In [17]:
# Get Not Applicable in Direction of Travel and drop these rows

not_app = motor_df[motor_df['Direction of Travel'] == 'Not Applicable'].index
motor_df.drop(not_app, inplace = True)

In [68]:
# Determining the dependent factor for the results of these contributing factors

directions = motor_df['Direction of Travel'].value_counts()
directions

East         94530
West         94191
South        92360
North        92221
Northeast    11517
Southwest    11124
Southeast    11071
Northwest    11061
Name: Direction of Travel, dtype: int64

In [69]:
# Determine wheather the accident is caused by humans or another factor

replace_direction = list(directions[directions < 12000].index)

# Replace in dataframe
for direction in replace_direction:
    motor_df["Direction of Travel"] = motor_df["Direction of Travel"].replace(direction, "Other")
    
# Check to make sure binning was successful
motor_df['Direction of Travel'].value_counts()

East     94530
West     94191
South    92360
North    92221
Other    44773
Name: Direction of Travel, dtype: int64

In [70]:
# Looking at the Vehicle Make column

motor_count = motor_df['Vehicle Make'].value_counts()
motor_count.head(15)

TOYOT    57239
HONDA    46095
CHEVR    44133
FORD     44126
NISSA    35334
JEEP     21089
HYUND    18128
DODGE    16350
SUBAR    13317
KIA       9343
GMC       8263
BMW       8175
ME/BE     7529
VOLKS     7428
CHRYS     7369
Name: Vehicle Make, dtype: int64

In [71]:
# Determine which values to replace if counts are less than 5000

replace_Vehicle = list(motor_count[motor_count < 3000].index)
replace_Vehicle
# Replace in dataframe
for car in replace_Vehicle:
    motor_df["Vehicle Make"] = motor_df["Vehicle Make"].replace(car,"Other")
    
# Check to make sure binning was successful
motor_df['Vehicle Make'].value_counts()

TOYOT    57239
HONDA    46095
CHEVR    44133
FORD     44126
NISSA    35334
Other    33098
JEEP     21089
HYUND    18128
DODGE    16350
SUBAR    13317
KIA       9343
GMC       8263
BMW       8175
ME/BE     7529
VOLKS     7428
CHRYS     7369
LEXUS     5752
ACURA     5745
MAZDA     5685
RAM       4640
BUICK     4562
INFIN     4461
MITSU     3586
AUDI      3495
CADIL     3133
Name: Vehicle Make, dtype: int64

In [73]:
# Look at Regristration Class value counts for binning

class_count = motor_df['Registration Class'].value_counts()
class_count.head(15)

PASSENGER OR SUBURBAN               347712
OMNIBUS - TAXI                       20924
AGRICULTURAL TRUCK                   14251
COMMERCIAL                            9818
SPECIAL PASSENGER                     5611
POLITICAL SUBDIVISION                 4352
MOTORCYCLE                            2771
INTERNATIONAL REGISTRATION            2037
OMNIBUS - REGULAR                     1742
ORGANIZATIONAL                        1687
OMNIBUS - LIVERY                      1528
OMNIBUS-SPECIAL (PRIVATE RENTAL)       896
TRACTOR - REGULAR                      618
TOW TRUCK                              534
SCHOOL CAR                             521
Name: Registration Class, dtype: int64

In [74]:
# Determine which values to replace if counts are less than 4000

replace_class = list(class_count[class_count < 2000].index)

# Replace in dataframe
for Class in replace_class:
    motor_df['Registration Class'] = motor_df['Registration Class'].replace(Class,"Other")
    
# Check to make sure binning was successful
motor_df['Registration Class'].value_counts()

PASSENGER OR SUBURBAN         347712
OMNIBUS - TAXI                 20924
AGRICULTURAL TRUCK             14251
Other                          10599
COMMERCIAL                      9818
SPECIAL PASSENGER               5611
POLITICAL SUBDIVISION           4352
MOTORCYCLE                      2771
INTERNATIONAL REGISTRATION      2037
Name: Registration Class, dtype: int64

In [75]:
# Action Prior to accident

actions = motor_df['Action Prior to Accident'].value_counts()
actions

Going Straight Ahead          243029
Making Left Turn               46153
Backing                        23642
Slowing or Stopping            22816
Making Right Turn              21186
Changing Lanes                 20355
Starting in Traffic             7221
Merging                         5984
Starting from Parking           5644
Overtaking/Passing              4366
Making U Turn                   4236
Entering Parked Position        3154
Other                           2636
Stopped in Traffic              2309
Parked                          2054
Avoiding Object in Roadway      1493
Unknown                         1307
Police Pursuit                   182
Making Right Turn on Red         177
Making Left Turn on Red           73
Not Applicable                    58
Name: Action Prior to Accident, dtype: int64

In [76]:
# Determine which values to replace if counts are less than 4000

replace_action = list(action[action < 2000].index)

# Replace in dataframe
for action in replace_action:
    motor_df['Action Prior to Accident'] = motor_df['Action Prior to Accident'].replace(action,"Other")
    
# Check to make sure binning was successful
motor_df['Action Prior to Accident'].value_counts()

Going Straight Ahead        243029
Making Left Turn             46153
Backing                      23642
Slowing or Stopping          22816
Making Right Turn            21186
Changing Lanes               20355
Starting in Traffic           7221
Merging                       5984
Other                         5926
Starting from Parking         5644
Overtaking/Passing            4366
Making U Turn                 4236
Entering Parked Position      3154
Stopped in Traffic            2309
Parked                        2054
Name: Action Prior to Accident, dtype: int64

In [78]:
# Recode Direction of Travel to integer: LinearRegression does not take string

def Classification(X):
    if X == 'East':
        return 0
    elif X == 'West':
        return 1
    elif X == 'South':
        return 2
    elif X == 'North':
        return 3
    else:
        return 4
    
motor_df['Direction of Travel'] = motor_df['Direction of Travel'].apply(Classification)

In [82]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME','Contributing Factor 1 Description',
# 'Contributing Factor 2 Description'.

motor_df = motor_df.drop(['Case Vehicle ID', 'Partial VIN', 'Contributing Factor 2'], axis=1)

In [18]:
motor_df.to_csv('Resources/Motor_Vehicle_Crashes_clean.csv')

In [83]:
# Generate our categorical variable lists
motor_cat = motor_df.dtypes[motor_df.dtypes == "object"].index.tolist()
motor_cat

['Vehicle Body Type',
 'Registration Class',
 'Action Prior to Accident',
 'Type / Axles of Truck or Bus',
 'Fuel Type',
 'State of Registration',
 'Vehicle Make',
 'Contributing Factor 1',
 'Contributing Factor 1 Description',
 'Contributing Factor 2 Description',
 'Event Type']

In [84]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list

encode_df = pd.DataFrame(enc.fit_transform(motor_df[motor_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(motor_cat)
encode_df.head()

,Vehicle Body Type_2 DOOR SEDAN,Vehicle Body Type_4 DOOR SEDAN,Vehicle Body Type_ALL TERRAIN VEHICLE,Vehicle Body Type_AMBULANCE,Vehicle Body Type_BUS (OMNIBUS),Vehicle Body Type_CEMENT MIXER,Vehicle Body Type_CONSTRUCTION EQUIPMENT,Vehicle Body Type_CONVERTIBLE,Vehicle Body Type_CUSTOM,Vehicle Body Type_DELIVERY TRUCK,...,"Event Type_Other*, Non-Collision","Event Type_Overturned, Non-Collision","Event Type_Pedestrian, Collision With","Event Type_Railroad Train, Collision With","Event Type_Ran Off Roadway Only, Non-Collision","Event Type_Sign Post, Collision With Fixed Object","Event Type_Snow Embankment, Collision With Fixed Object","Event Type_Submersion, Non-Collision","Event Type_Tree, Collision With Fixed Object",Event Type_Unknown
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
# Merge one-hot encoded features and drop the originals

motor_df = motor_df.merge(encode_df, left_index=True,right_index=True)
motor_df = motor_df.drop(motor_cat, 1)
motor_df.head()

,Year,Direction of Travel,Vehicle Year,Number of Occupants,Engine Cylinders,Vehicle Body Type_2 DOOR SEDAN,Vehicle Body Type_4 DOOR SEDAN,Vehicle Body Type_ALL TERRAIN VEHICLE,Vehicle Body Type_AMBULANCE,Vehicle Body Type_BUS (OMNIBUS),...,"Event Type_Other*, Non-Collision","Event Type_Overturned, Non-Collision","Event Type_Pedestrian, Collision With","Event Type_Railroad Train, Collision With","Event Type_Ran Off Roadway Only, Non-Collision","Event Type_Sign Post, Collision With Fixed Object","Event Type_Snow Embankment, Collision With Fixed Object","Event Type_Submersion, Non-Collision","Event Type_Tree, Collision With Fixed Object",Event Type_Unknown
3,2018,2,2013.0,5.0,4.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,2018,1,2007.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,2018,3,2013.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,2018,2,2014.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,2018,0,2006.0,3.0,6.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
# Split our preprocessed data into our features and target arrays

y = motor_df['Direction of Travel'].values
X = motor_df.drop("Direction of Travel", axis = 1).values

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [87]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg').fit(X_train, y_train)
preds = model1.predict(X_test)

C:\Users\Patrick\anaconda3\envs\mlenv\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Patrick\anaconda3\envs\mlenv\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Patrick\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [88]:
preds
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, preds))

0.22499075785582254


In [89]:
from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, preds)
print(matrix)

[[890 875 488 793   4]
 [861 858 528 792   3]
 [848 849 525 737   8]
 [859 836 522 764   5]
 [389 450 238 397   6]]


In [90]:
report = classification_report(y_test, preds)
print(report)

              precision    recall  f1-score   support

           0       0.23      0.29      0.26      3050
           1       0.22      0.28      0.25      3042
           2       0.23      0.18      0.20      2967
           3       0.22      0.26      0.24      2986
           4       0.23      0.00      0.01      1480

    accuracy                           0.22     13525
   macro avg       0.23      0.20      0.19     13525
weighted avg       0.23      0.22      0.21     13525

